# Practice: Spotify's API

This notebook contains a set of exercises that will guide you through the different steps of this practice exercise. Devise solutions that are code-based, i.e. not hard-coded or manually computed. 

<div class="alert alert-success">The aim of this exercise is to create and save a dataset containing information about every song in a given playlist by requesting data from Spotify's API.</div>

## Getting client credentials

Spotify's API uses an Authentication scheme called OAuth. Hence, before starting to make requests, you need to get your client credentials to the Spotify API. 

To do so, you need to have a Spotify account (free or paid). If you don't have one yet, please create a free account before moving on. Once you do, head over to Spotify for Developers, open your [Dashboard](https://developer.spotify.com/dashboard/) and log in with your account. 

Click on “CREATE APP”. Fill in the name and description for your project. Under "Redirect URI", enter "http://localhost:8888/callback" (don't worry about this, as it won't be used). Finally, it the tick boxes, select "Web API". 

Once your App has been created, you should see the app name in the Dashboard. Click on it, then on "Settings", and then on "View Client Secret".  The numbers for “Client ID” and “Client Secret” on correspond to your client credentials.

<div class="alert alert-info">Create two new variables, <b>client_id</b> and <b>client_secret</b>, that store your ID and Key, respectively</div>

In [30]:
client_id = "94f61c00d73e479f9127b55fc83e68d8"
client_secret = "610ee0f6129d44649b7ff5449c3084dd"

These are the keys that you will use throughout the remaining of the assignment.



Great! We are good to go. Next step is getting an access token.

## Getting an access token / key

In order to access the various endpoints of the Spotify API, we need to pass an access token. 

To get one, we need to pass a ```POST``` request with our client credentials. This request will create a token resource in the server and respond back with it. We can build this ```POST``` request using ```requests``` library. In other words, the first API call in the spotify API is the call to get the access token. 

<div class="alert alert-info">Run the following cell to built your POST request</div>

In [31]:
import requests

# URL for token resource
auth_url = 'https://accounts.spotify.com/api/token'

# request body
params = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}
        #   'market': "ES"}

# POST the request
response =  requests.post(auth_url, data=params)
token_response = response.json()
token_response

{'access_token': 'BQDFH_3Nr-KEFASHWw-RTfUcOgTtiXybGpVAejKLYgeAaMnOhMTgu5QXhIiXvzPPMBnYX7IGWkvgGJiS9PlXzlHcLkDFhoeSB8ck6nvYRbYywBNGbuY',
 'token_type': 'Bearer',
 'expires_in': 3600}

Take a look at the response you obtained. The access_token you just retrieved will expire after one hour.

<div class="alert alert-info">Retrieve your token from <b>auth_response</b> and save it in a new variable called <b>access_token</b>. </div>

<div class="alert alert-warning">Make sure you define the <i>access_token</i> variable such that it will be updated each time your code is run from scratch, i.e. make sure it hasn't expired by the time your code is graded.</div>

In [32]:
access_token = token_response["access_token"]
access_token

'BQDFH_3Nr-KEFASHWw-RTfUcOgTtiXybGpVAejKLYgeAaMnOhMTgu5QXhIiXvzPPMBnYX7IGWkvgGJiS9PlXzlHcLkDFhoeSB8ck6nvYRbYywBNGbuY'

As above, the access token will be used throughout the remaining of the assignment.



This token is your golden ticket to access Spotify's API. A copy of this string is now stored in the server, so that everytime you make a request to the API the server will check that the token you provide and the one it has in store match.

<img src="https://www.dropbox.com/s/hgb02k4h1mtdv22/header.png?raw=1" width="500">

Similar to the Cohere API, Spotify's API expects you to include your access token in the requests header. We won't use headers in this course and this is the only exception: when some APIs require to specify the authentication in the header. Hence, the header has already been formatted for you. 

<div class="alert alert-info">Run the following cell to save the header in a new variable so that you can use it later on.</div>

In [33]:
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

## Poking around

Spotify's API provides numerous endpoints to access things like album listings, artist information, playlists, even Spotify-generated audio analysis of individual tracks, which include their time signature or measurements such as their “danceability” or "loudness". You can take a look at all the information available by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). In this assignment you will use several of these endpoints.

In order to get a feel of how the API works, we will begin by making a ```GET``` request to the ```audio-features``` endpoint to extract data for a specific track. In particular, let's retrieve all the information for Radiohead's *Creep* song. 

The first thing you need is to identify the appropriate URL or path to direct your request to. The urls for all Spotify API endpoints follow the same structure. They all use the base URL for the API and are then defined as a concatenation of ```base_url + endpoint```. Sometimes, you will also need to provide some additional information as part of the URL. In the case of ```audio-features```, however, it is enough with just the ```base_url``` and the ```endpoint``` name.

The ```base_url``` is defined below:

In [34]:
base_url = 'https://api.spotify.com/v1'

<div class="alert alert-info">Define the url for the audio-features endpoint by following the instructions above. Store it in a variable called <b>audio_features_endpoint</b>.</div>

In [35]:
audio_features_endpoint = "audio-features"

Next thing we need is to fill in the request body. If you check the documentation you'll see that the ```audio-features``` endpoint takes the following query parameters.

<img src="https://www.dropbox.com/s/s4zs6wlue0u16cu/body.png?raw=1" width="500">

Hence, the final thing you need to extract data about Radiohead's Creep song is to locate its ```id```. This is its unique identifier. Spotify has unique ids for tracks, for artists, for albums, for playlists, etc.

![Creep](https://www.dropbox.com/s/kufj6ww2yn069gb/creep.png?raw=1)

You can get the ```id``` for any song by going to Spotify, looking for the song, clicking the “…” by the song name, then “Share” and then “Copy Spotify URI”. 

<div class="alert alert-warning">Note that this procedure also works for retrieving ids for artists, albums or any other resource type. Alternatively, you can also retrieve the id for a song or a playlist by clicking "..." by the name, then "Share" and the "Copy Link". This will return a full link to the resource. The id corresponds to the string right after the last slash sign / and before the ? sign</div>

This URI should be a string that includes something like **spotify:track:**, followed by an alphanumeric sequence. This sequence is the ID you are looking for.

<div class="alert alert-info">Create a new variable called <b>track_id</b> that stores the ID for Radiohead's song Creep.</div>

In [36]:
track_id = "70LcF31zb1H0PyJoS1Sx1r?si=e629191aa6674191"

Now that you have the id, let's format the URL of the request and send the actual GET request to retrieve the data:

In [37]:
URL = f"{base_url}/{audio_features_endpoint}/{track_id}"

Now that everything is ready, you can send the actual GET request to retrieve the data.

<div class="alert alert-info">Remember to pass the <i>url</i>, the <i>headers</i> and the <i>params</i> dictionary as arguments to the <i>get</i> function. Convert the response to <i>json</i> format and store it in a new variable called <b>creep</b>.</div>

In [38]:
response = requests.get(URL, headers=headers, params=params)
creep = response.json()
creep

{'danceability': 0.515,
 'energy': 0.43,
 'key': 7,
 'loudness': -9.935,
 'mode': 1,
 'speechiness': 0.0372,
 'acousticness': 0.0097,
 'instrumentalness': 0.000133,
 'liveness': 0.129,
 'valence': 0.104,
 'tempo': 91.844,
 'type': 'audio_features',
 'id': '70LcF31zb1H0PyJoS1Sx1r',
 'uri': 'spotify:track:70LcF31zb1H0PyJoS1Sx1r',
 'track_href': 'https://api.spotify.com/v1/tracks/70LcF31zb1H0PyJoS1Sx1r',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70LcF31zb1H0PyJoS1Sx1r',
 'duration_ms': 238640,
 'time_signature': 4}


<div class="alert alert-warning">Often Spotify has several instances of a track in its catalogue, each available in a different set of markets. This commonly happens when the track the album is on has been released multiple times under different licenses in different markets. These tracks are linked together so that when a user tries to play a track that isn’t available in their own market, the Spotify mobile, desktop, and web players try to play another instance of the track that is available in the user’s market. Even if this issue won't prevent you from being able to play the  song in your device, it might result in different ids for the different markets. Note that the test below is written taking the values for the Spanish market into account.</div>

<div class="alert alert-warning">Note that you can specify the market for this and for future queries through the <i>market</i> parameter.</div>

Congrats! You just made your first successful request to Spotify's API! Feel free to take a look at the information included in the response. Pay special attention to the way in which information is presented. Once you are done, let's move on to some actual work!

## Getting data from a playlist

In the following exercise you will build a dataset containing data about different songs. You can either use a playlist of your own, or use the one mentioned in the documentation.


<div class="alert alert-info">Create a variable called <b>playlist_id</b> that stores the id of your playlist of choice.</div>

<div class="alert alert-warning">Your <i>playlist_id</i> should contain only alphanumeric characters. This means that characters such as ? $ % & / ! should not be included.</div>

In [39]:
playlist_id = "2yT5G3PPy1h7MyPQ4gNme6"

In the following you are going to extract information about the different tracks included in this playlist. Hence, let's make sure that the variabl ei spropelry created and that it refers to an actual id.

The next step will be making a request to get full details of the tracks included in your chosen playlist. Remember that you can take a look at all the information available at the different endpoints in Spotify's API by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). Locate the right endpoint for your query and read the Docs to find out how to build your request. 

<div class="alert alert-info"><b>Exercise 1 </b>Write the code to retrieve all the items from your chosen playlist from the <i>tracks</i> endpoint. When making your request don't use any of the optional arguments. Direct your request to the right endpoint instead. Store the <i>raw</i> response in a new variable called <i>playlist_response</i>.
</div>

<div class="alert alert-warning">To complete this exercise you must use the requests library</div>

In [40]:
playlist_endpoint = "playlists"
URL = f"{base_url}/{playlist_endpoint}/{playlist_id}"

In [41]:
playlist_response = requests.get(URL, headers=headers, params=params)

<div class="alert alert-info"><b>Exercise 2 </b>Convert the response to JSON format and store the result in a new variable called <i>playlist</i>.</div>

In [42]:
playlist = playlist_response.json()

The following cells contain the tests that will grade your code. **Don't modify them**. Simply leave them as they are.

Take your time to familiarize yourself with the data and how they are presented. Note that, by default, Spotify's API only returns information about a maximum of 100 tracks in a playlist. If your playlist of choice has more that 100 tracks, you'll retrieve the data only for the first 100 of them.

<div class="alert alert-danger">Throghout the following exercises you may come across data that are missing. If so, encode these data using a <b>None</b> (Nonetype), unless otherwise stated. </div>

## Retrieving basic track information

In what follows, you are going to retrieve specific data for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 3 </b>Write the code to extract the title (in string form), the name of the album (in string form), the name of the artist (in string form), the duration (in integer form), the track number (in integer form), the popularity (in integer form), and the id (in string form) for all the tracks included in your chosen playlist. Store these data in separate lists called <i>title</i>, <i>album</i>, <i>artist</i>, <i>duration</i>, <i>track_number</i>, <i>track_popularity</i>, and <i>track_id</i>, respectively. In those cases where more than one value is available for these items, retain only the first. In all cases, entries should appear in the same order as they are presented in the playlist.<br></div>

<div class="alert alert-warning">Make sure you correctly set all variable names. They have to be written <b>exactly</b> as given in the instructions.</div>

In [43]:
title = []
album = []
artist = []
duration = []
track_number = []
track_popularity = []
track_id = []

playlist_tracks = playlist["tracks"]

for i in playlist_tracks["items"]:
    track = i["track"]

    title.append(track["name"])
    album.append(track['album']['name'])
    artist.append(track['artists'][0]['name'])  
    duration.append(track['duration_ms'])
    track_number.append(track['track_number'])
    track_popularity.append(track['popularity'])
    track_id.append(track['id'])

<div class="alert alert-info"><b>Exercise 4 </b>Write the code to extract the number of available markets (in integer form) for all the tracks included in your chosen playlist. Store these data in a separate list called <i>n_available_markets</i>. As above, entries in this list should appear in the same order as they are presented in the playlist.</div>

In [44]:
n_available_markets = []

for i in playlist_tracks["items"]:
    track = i["track"]

    n_available_markets.append(len(track["available_markets"]))

Each track in the playlist is associated to an album. Let's retrieve the corresponding release date.

<div class="alert alert-info"><b>Exercise 5 </b>Write the code to extract the release year (in int form) and month (in string form) for all the tracks included in your chosen playlist. Store these data in separate lists called <i>release_year</i> and <i>release_month</i>, respectively. Entries should appear in these lists in the same order as they are presented in the playlist.</div>

<div class="alert alert-warning">Make sure you correctly set the variable names. They have to be written <b>exactly</b> as given in the instructions.</div>

<div class="alert alert-warning">Months should be stored with their full names and capitalized: use <i>September</i> for 09, <i>January</i> for 01, etc.</div>

In [45]:
from datetime import datetime

release_year = []
release_month = []

for i in playlist_tracks["items"]:
    
    track = i["track"]

    if track["album"]["release_date_precision"] == "day":
        date = track["album"]["release_date"]
        year, month, day = date.split("-")
        release_year.append(int(year))
        month_name = datetime.strptime(month, '%m').strftime('%B')
        release_month.append(month_name)

    elif track["album"]["release_date_precision"] == "year":
        date = track["album"]["release_date"]
        release_year.append(int(year))
        release_month.append(None)

    elif track["album"]["release_date_precision"] == "month":
        date = track["album"]["release_date"]
        year, month = date.split("-")
        release_year.append(int(year))
        month_name = datetime.strptime(month, '%m').strftime('%B')
        release_month.append(month_name)

## Retrieving additional track information

In what follows, you are going to retrieve additional data for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 6 </b>Write the code to extract data about the danceability, energy, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence and tempo for all the tracks included in your chosen playlist. Store these data in separate lists called <i>danceability</i> (in float form), <i>energy</i>, <i>loudness</i> (in float form), <i>mode</i> (in int form), <i>speechiness</i> (in float form), <i>acousticness</i> (in float form), <i>instrumentalness</i> (in int and float form), <i>liveness</i> (in float form), <i>valence</i> (in float form) and <i>tempo</i> (in float form), respectively. In those cases where more than one value is available for these items, retain only the first. In all cases, entries should appear in the same order as they are presented in the playlist.</div>

In [46]:
danceability = []
energy = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

for i in playlist_tracks["items"]:
    track = i["track"]
    track_id = track["id"]
    URL = f"{base_url}/{audio_features_endpoint}/{track_id}"

    track_response = requests.get(URL, headers=headers, params=params)
    track_response = track_response.json()

    if isinstance(track_response["danceability"], list):
        danceability.append(track_response["danceability"][0])
    else:
        danceability.append(track_response["danceability"])

    if isinstance(track_response["energy"], list):
        energy.append(track_response["energy"][0])
    else:
        energy.append(track_response["energy"])

    if isinstance(track_response["loudness"], list):
        loudness.append(track_response["loudness"][0])
    else:
        loudness.append(track_response["loudness"])

    if isinstance(track_response["mode"], list):
        mode.append(track_response["mode"][0])
    else:
        mode.append(track_response["mode"])

    if isinstance(track_response["speechiness"], list):
        speechiness.append(track_response["speechiness"][0])
    else:
        speechiness.append(track_response["speechiness"])

    if isinstance(track_response["acousticness"], list):
        acousticness.append(track_response["acousticness"][0])
    else:
        acousticness.append(track_response["acousticness"])

    if isinstance(track_response["instrumentalness"], list):
        instrumentalness.append(track_response["instrumentalness"][0])
    else:
        instrumentalness.append(track_response["instrumentalness"])

    if isinstance(track_response["liveness"], list):
        liveness.append(track_response["liveness"][0])
    else:
        liveness.append(track_response["liveness"])

    if isinstance(track_response["valence"], list):
        valence.append(track_response["valence"][0])
    else:
        valence.append(track_response["valence"])

    if isinstance(track_response["tempo"], list):
        tempo.append(track_response["tempo"][0])
    else:
        tempo.append(track_response["tempo"])

## Retrieving artist information

In what follows, you are going to retrieve data about the artists for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 7 </b>Write the code to extract data about the total number of followers (in int form), the first listed genre (in string form) and the popularity (in int form) for the artists of all the tracks included in your chosen playlist. Store these data in separate lists called <i>artist_followers</i>, <i>genre</i> and <i>artist_popularity</i>. In cases where no genre is given, fill in the corresponding entry using a None.</div>

<div class="alert alert-warning">Note that the artist information has to to be extracted from the tracks themselves.</div>

In [47]:
artists_endpoint = "artists"

In [48]:
playlist_tracks["items"][0]["track"]["artists"]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/5Pb27ujIyYb33zBqVysBkj'},
  'href': 'https://api.spotify.com/v1/artists/5Pb27ujIyYb33zBqVysBkj',
  'id': '5Pb27ujIyYb33zBqVysBkj',
  'name': 'RÜFÜS DU SOL',
  'type': 'artist',
  'uri': 'spotify:artist:5Pb27ujIyYb33zBqVysBkj'}]

In [49]:
artist_followers = []
genre = []
artist_popularity = []

for i in playlist_tracks["items"]:
    track = i["track"]
    for artist in track["artists"]:
        artist_id = artist["id"]
        URL = f"{base_url}/{artists_endpoint}/{artist_id}"

        artist_response = requests.get(URL, headers=headers, params=params)
        artist_response = artist_response.json()

        #followers
        artist_followers.append(int(artist_response["followers"]["total"]))
        
        # genre
        if len(artist_response["genres"]) == 0:
            genre.append(None)
        else:
            genre.append(artist_response["genres"][0])

        # popularity
        artist_popularity.append(int(artist_response["popularity"]))


In [50]:
print(artist_followers)
print(genre)
print(artist_popularity)

[1840068, 734996, 734996, 713852, 10223909, 7496977, 1507248, 636657, 548450, 5828894, 636657, 668059, 668059, 5828894, 734996, 88161, 986069, 958808, 475331, 10255909, 10255909]
['australian electropop', 'electronica', 'electronica', 'progressive metal', 'electro', 'funk', 'ambient', 'downtempo', 'melodic house', 'new wave', 'downtempo', 'electronica', 'electronica', 'new wave', 'electronica', 'destroy techno', 'german techno', 'downtempo', 'classic dubstep', 'atl hip hop', 'atl hip hop']
[77, 58, 58, 55, 82, 72, 67, 63, 68, 78, 63, 58, 58, 78, 58, 43, 60, 64, 51, 81, 81]


In addition to the above, there's also additional information we can retrieve about each artist. For this purpose, let's first retrieve the list of distinct artists for the different tracks in your playlist.

<div class="alert alert-info"><b>Exercise 8 </b>Write the code to identify the list of unique artist ids that correspond to the different tracks in your chosen playlist. Store these data in a list called <i>unique_artist_ids</i>.</div>

<div class="alert alert-warning">Retrieve the artist ids that correspod directly to the tracks.</div>

In [51]:
artists_ids = []

for i in playlist_tracks["items"]:
    track = i["track"]
    for artist in track["artists"]:
        artist_id = artist["id"]
        artists_ids.append(artist_id)
        
artists_ids_unique = set(artists_ids)


We are now interested in retrieving catalog information about each artist’s top tracks. This information is provided by Spotify's API on a country basis. Here, we will retrieve the information corresponding to Spain, whose *ISO 3166-1 alpha-2* code is **ES**. The information we are looking for is stored under the ```top-tracks``` endpoint for ```artists```. Requests to this location retrieve the 10 most famous tracks for a given artist id.

<div class="alert alert-info"><b>Exercise 9 </b>Write the code to retrieve the 10 top tracks for each of the unique artists in your chosen playlist. Store these data in a dictionary called <i>top_tracks</i>. The keys of this dictionary should correspond to the unique artist ids stored in the list <i>unique_artist_id</i>. The values of this dictionary should include the names of the 10 most popular songs in a list.</div>

<div class="alert alert-warning">In cases where the number of tracks included in the top-tracks endpoint is below 10, simply retrieve the provided list.</div>


In [52]:
top_tracks_endpoint = "top-tracks"

top_tracks = {}

for artist in artists_ids_unique:
    URL = f"{base_url}/{artists_endpoint}/{artist_id}/{top_tracks_endpoint}"

    artist_top10_response = requests.get(URL, headers=headers, params=params)
    artist_top10_response = artist_top10_response.json()

    l = []

    for top_track in artist_top10_response["tracks"]:
        l.append(top_track["name"])
    
    top_tracks[artist] = l

In [53]:
top_tracks_endpoint = "top-tracks"

top_tracks = {}

for artist in artists_ids_unique:
    URL = f"{base_url}/{artists_endpoint}/{artist}/{top_tracks_endpoint}" # esta línea estaba mal en la entrega que hice

    artist_top10_response = requests.get(URL, headers=headers, params=params)
    artist_top10_response = artist_top10_response.json()

    l = []

    for top_track in artist_top10_response["tracks"]:
        l.append(top_track["name"])
    
    top_tracks[artist] = l

We can now use this information to identify those songs in your chosen playlist that correspond to each artist's top tracks.

<div class="alert alert-info"><b>Exercise 10 </b>Write the code to check whether the different tracks in your chosen playlist are included among the corresponding artist's top tracks. Store the results in a list called <i>is_top</i>. This list should include the boolean value <i>True</i> whenever the considered track is among the top tracks for that artist and <i>False</i> otherwise.</div>

<div class="alert alert-warning">When comparing track names, consider only <b>exact</b> matches.</div>

In [62]:
is_top = []

for j in playlist_tracks["items"]:
    track_name = j["track"]["name"]
    for artist in j["track"]["artists"]:
        artist_id = artist["id"]
        if artist_id in top_tracks.keys():
            top_track_names = top_tracks[artist_id]
            is_top.append(track_name in top_track_names)

21

A very interesting feature in Spotify is the song recommender.

When creating a playlist from scratch, Spotify offers help by suggesting additional tracks that may be related to the ones already added. Spotify's API also provides a specific endpoint for `recommendations`. We will use this endpoint to retrieve one recommendation for each song in our original playlist.

<div class="alert alert-info"><b>Exercise 11 </b>Write the code to retrieve the id, title and artist from the list of recommendations for each track in your playlist. Store these data in new lists called <i>recommendatin_id</i>, <i>recommendation_title</i> and <i>recommendation_artist</i>.</div>

<div class="alert alert-warning">Retrieve recommendations based solely on each track.</div>

<div class="alert alert-warning">Retrieve information only about the first track in the recommendations list.</div>

<div class="alert alert-warning">When more than one artist is associated to the recommended track, retrieve information only about the first.</div>

In [57]:
recommendations_endpoint = "recommendations"

recommendation_id = []
recommendation_title = []
recommendation_artist = []

for i in playlist_tracks["items"]:
    track_id = i["track"]["id"]

    URL = f"{base_url}/{recommendations_endpoint}/?seed_tracks={track_id}"
    recommendation_response = requests.get(URL, headers=headers, params=params)
    recommendation = recommendation_response.json()

    track_recommendation = recommendation["tracks"][0]
    recommendation_id.append(track_recommendation["id"])
    recommendation_title.append(track_recommendation["name"])
    recommendation_artist.append(track_recommendation["artists"][0]["name"])

## Bonus exercise

You can complete the exercises below to obtain extra points. These points will be added to your score directly. The maximum score in this assignment is a 10.

<div class="alert alert-danger"><b>Bonus 1 </b>Write the code to find the most popular song in your playlist. If several songs have the same popularity, choose the one for which the artist has the most followers. Save the name of the song to a new variable called <i>most_popular</i>. This variable should <b>only</b> contain the name of the most popular song in string form.</div>

In [58]:
popularities = {}
popularities["track_name"] = []
popularities["track_id"] = []
popularities["track_popularity"] = []

for i in playlist_tracks["items"]:
    track = i["track"]
    track_id = track["id"]
    track_name = track["name"]


    track_popularity = track['popularity']
    # popularities[(track_name, track_id)] = track_popularity
    popularities["track_name"].append(track_name)
    popularities["track_id"].append(track_id)
    popularities["track_popularity"].append(track_popularity)

import pandas as pd
popularities = pd.DataFrame(popularities)
popularities.sort_values("track_popularity", ascending=False).iloc[0]
most_popular = popularities.sort_values("track_popularity", ascending=False).loc[0,"track_name"] # esta líne a está mal, debería haber utilizado iloc de nuevo.

You might have noticed that there exist slight incosistencies in the way in which the track title are included in the playlist, versus the top 10 artist tracks. For example, the fourht song in the provided playlist appears as <i>High And Dry</i> within the title of the playlist songs but as <i>High and Dry</i> (with a small "a" for the word "and") in the Radiohead's top tracks. This results in the fact that the song of the playlist will not be included as one of the artist's top tracks in the is_top list.

<div class="alert alert-danger"><b>Bonus 2 </b>Write the code to create a new list called <i>is_top_2</i>. This time make sure that you look for partial matches of title names too.</div>

In [59]:
is_top = []

def normalize_string(s):
    special_characters = [
    ".", ",", ":", ";", "!", "?", "'", "\"", "-", "_", "...", "(", ")", "[", "]", "{", "}",
    "+", "-", "*", "/", "%", "=", "<", ">", "&", "|", "^", "~", "#", "$", "@",
    "\\", "/", "\n", "\t", ":", "°", "©", "®", "™", "§", "·",
    "á", "é", "í", "ó", "ú", "ñ", "ü",
    "€", "£", "¥", "₹", "₿", "¢",
    "==", "!=", "<=", ">="
    ]

    s = s.lower()
    s = s.replace(" ","")
    for char in special_characters:
        s = s.replace(char,"")
    
    return s

for j in playlist_tracks["items"]:
    track_name = j["track"]["name"]
    for artist in j["track"]["artists"]:
        artist_id = artist["id"]
        if artist_id in top_tracks.keys():
            top_track_names = top_tracks[artist_id]
            top_track_names = [normalize_string(item) for item in top_track_names]
            track_name = normalize_string(track_name)
            is_top.append(track_name in top_track_names)